# Question B2 (10 marks)
In Question B1, we used the Category Embedding model. This creates a feedforward neural network in which the categorical features get learnable embeddings. In this question, we will make use of a library called Pytorch-WideDeep. This library makes it easy to work with multimodal deep-learning problems combining images, text, and tables. We will just be utilizing the deeptabular component of this library through the TabMlp network:

In [1]:
!pip install pytorch-widedeep

In [2]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from pytorch_widedeep.preprocessing import TabPreprocessor
from pytorch_widedeep.models import TabMlp, WideDeep
from pytorch_widedeep import Trainer
from pytorch_widedeep.metrics import R2Score

1.Divide the dataset (‘hdb_price_prediction.csv’) into train and test sets by using entries from the year 2020 and before as training data, and entries from 2021 and after as the test data.

In [4]:
df = pd.read_csv('hdb_price_prediction.csv')

# Training Data
df_train = df[df['year'] <= 2020].copy()
# Testing Data
df_test = df[df['year'] >= 2021].copy()

# Dropping Unncessary Columns
df_train.drop(columns=['year','full_address','nearest_stn'], inplace=True)
df_test.drop(columns=['year','full_address','nearest_stn'], inplace=True)

print("Training Data:", df_train.shape)
print("Testing Data:", df_test.shape)

Training Data: (87370, 11)
Testing Data: (72183, 11)


2.Refer to the documentation of Pytorch-WideDeep and perform the following tasks:
https://pytorch-widedeep.readthedocs.io/en/latest/index.html
* Use [**TabPreprocessor**](https://pytorch-widedeep.readthedocs.io/en/latest/examples/01_preprocessors_and_utils.html#2-tabpreprocessor) to create the deeptabular component using the continuous
features and the categorical features. Use this component to transform the training dataset.
* Create the [**TabMlp**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/model_components.html#pytorch_widedeep.models.tabular.mlp.tab_mlp.TabMlp) model with 2 linear layers in the MLP, with 200 and 100 neurons respectively.
* Create a [**Trainer**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/trainer.html#pytorch_widedeep.training.Trainer) for the training of the created TabMlp model with the root mean squared error (RMSE) cost function. Train the model for 100 epochs using this trainer, keeping a batch size of 64. (Note: set the *num_workers* parameter to 0.)

In [7]:
# Define continuous and categorical columns
continuous_cols = ['dist_to_nearest_stn', 'dist_to_dhoby', 'degree_centrality',
                   'eigenvector_centrality', 'remaining_lease_years', 'floor_area_sqm']
cat_embed_cols = ['month', 'town', 'flat_model_type', 'storey_range']

In [8]:
# Create and fit the TabPreprocessor
tab_preprocessor = TabPreprocessor(
    cat_embed_cols = cat_embed_cols, continuous_cols = continuous_cols
)

# Scaled Training Data
X_tab = tab_preprocessor.fit_transform(df_train)

C:\Users\Admin\anaconda3\envs\myenv_py38\lib\site-packages\pytorch_widedeep\preprocessing\tab_preprocessor.py:360: UserWarning: Continuous columns will not be normalised
  warnings.warn("Continuous columns will not be normalised")


In [10]:
tab_mlp = TabMlp(column_idx=tab_preprocessor.column_idx,
                 cat_embed_input=tab_preprocessor.cat_embed_input,
                 cat_embed_dropout=0.1,
                 continuous_cols=continuous_cols,
                 mlp_hidden_dims=[200, 100]) # Two linear layers with 200 and 100 neurons

wide_deep = WideDeep(deeptabular=tab_mlp)

In [11]:
# Create the Trainer
trainer = Trainer(
    model=wide_deep,
    objective="regression",
    metrics=[R2Score],
    batch_size=64,
    num_workers=0
)

In [12]:
# Train the model
trainer.fit(
    X_tab=X_tab,
    target=df_train['resale_price'],
    n_epochs=100,
)

epoch 27:   0%|          | 0/2731 [00:00<?, ?it/s, loss=2.46e+9, metrics={'r2': 0.8861}]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

epoch 49:  49%|████▉     | 1338/2731 [00:04<00:04, 333.64it/s, loss=2.48e+9, metrics={'r2': 0.8954}]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

epoch 73:  25%|██▍       | 675/2731 [00:02<00:06, 323.63it/s, loss=2.44e+9, metrics={'r2': 0.8922}]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending

3.Report the test RMSE and the test R2 value that you obtained.

In [21]:
# Scaled Test Data
X_test = tab_preprocessor.transform(df_test)

# Make predictions on the test dataset
y_pred = trainer.predict(X_tab = X_test)

predict: 100%|██████████| 2256/2256 [00:02<00:00, 888.38it/s]


In [22]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

y_true = df_test['resale_price']

mse = mean_squared_error(y_true, y_pred)
print("Root Mean Squared Error (RMSE):", np.sqrt(mse))
r2 = r2_score(y_true, y_pred)
print("R2 Score:", r2)

Root Mean Squared Error (RMSE): 94274.93646960524
R2 Score: 0.6894789636373253
